In [1]:
import requests
import json
import base64

import pandas as pd

In [15]:
def read_json(filename):
    with open(filename) as f:
        return json.loads(f.read())

In [16]:
cred_filename = "./twitter_credentials.json"
credentials = read_json(cred_filename)

search_url = "https://api.twitter.com/1.1/search/tweets.json"
auth_url = "https://api.twitter.com/oauth2/token"

# Authentication

In [3]:
def get_bearer_token(consumer_key, consumer_secret):
    # Not sure why this needs all the encoding.
    key_secret = "{}:{}".format(consumer_key, consumer_secret)
    b64_encoded_key = base64.b64encode(key_secret.encode("ascii")).decode("ascii")

    headers={
        "Authorization": 'Basic {}'.format(b64_encoded_key), 
        "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8",
    }

    resp = requests.post(auth_url, headers=headers, data={'grant_type': 'client_credentials'})
    token = resp.json()["access_token"]
    
    return token

In [4]:
access_token = get_bearer_token(credentials["CONSUMER_KEY"], credentials["CONSUMER_SECRET"])

# Get Tweets

In [11]:
def search_tweets(query, count, result_type="recent"):
    search_headers = {
        'Authorization': 'Bearer {}'.format(access_token)    
    }

    search_params = {
        'q': query,
        'count': count,
        'result_type': result_type,
        "include_entities": False,
    }

    js = requests.get(search_url, headers=search_headers, params=search_params).json()["statuses"]
    tweets = pd.DataFrame(js, columns=["created_at", "id", "lang", "retweet_count", "text", "user"])
    tweets["user"] = tweets["user"].str["screen_name"]
    tweets["created_at"] = pd.to_datetime(tweets["created_at"], infer_datetime_format=True)
    
    return tweets

In [12]:
tweets = search_tweets("trump", 10)

In [13]:
tweets

,created_at,id,lang,retweet_count,text,user
0,2018-12-21 17:09:00,1076162635031146496,de,0,Wie wäre es mit einem Buch als Weihnachtsgesch...,Blog1600Penn
1,2018-12-21 17:09:00,1076162634989428736,en,7,RT @Realmommabear: @ChuckGrassley We don't dis...,Betsy_Manning
2,2018-12-21 17:09:00,1076162634846629888,en,2341,RT @SaysHummingbird: MAKE. THIS. GO. VIRAL. 🔥 ...,pkraayenbrink
3,2018-12-21 17:09:00,1076162634800685056,en,540,RT @RealJack: BACKSTAB: McCain Leaked Phony Do...,KWhittler
4,2018-12-21 17:09:00,1076162634683052032,en,635,RT @maddow: “Once again we see a president who...,ReneeVoiceBrand
5,2018-12-21 17:09:00,1076162634486083585,en,553,RT @JRubinBlogger: We have gotten to this sorr...,beebecathy
6,2018-12-21 17:09:00,1076162634481836033,en,1742,RT @joncoopertweets: Trump should resign immed...,woodysnoop
7,2018-12-21 17:09:00,1076162634431582210,en,3694,RT @ChelseaClinton: Merry Christmas from Presi...,MKJayAtWork
8,2018-12-21 17:09:00,1076162634016268288,en,1317,"RT @RealJamesWoods: ""’The Trump temper tantrum...",Colleen0707
9,2018-12-21 17:09:00,1076162633940647937,en,158,RT @peterdaou: GOP LEADERS ARE TRUMP COLLABORA...,IntrovertDeb
